Import the required python packages


In [ ]:
#  !pip install tsplib95
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt,tsplib95,multiprocessing,threading

## Create necessary classes and functions to build the TSP problem setup

Class to handle "TSP cities"

In [ ]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

Create a fitness function that ranks the TSP individuals in a given population. In this case, the individuals of the population are the different possible TSP paths

In [ ]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0
    
    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

## Create our initial population

Route generator

In [ ]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return route

Create first "population" (list of routes)

In [ ]:
def initialPopulation(popSize, cityList):
    population = []

    for i in range(0, popSize):
        population.append(createRoute(cityList))
    return population

Create N subpopulations from population (list of routes)

In [ ]:
def createSubPopulation(popSize,cityList, subPopSize):
  subPopulation = []

  for i in range(0,subPopSize):
    subPopulation.append(initialPopulation(popSize,cityList))
  return subPopulation

## Create the genetic algorithm

Rank individuals

In [ ]:
def rankRoutes(population): #Double Arr
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

Sort routes in ascending order

In [ ]:
def rankRoutesAsc(population): #Double Arr
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1))

Create a selection function that will be used to make the list of parent routes

In [ ]:
def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

Create mating pool

In [ ]:
def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

Create a crossover function for two parents to create one child. We are specifically using ordered crossover to maintain the sequence of the cities in a route.

In [ ]:
def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])
        
    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child

Create function to run crossover over full mating pool

In [ ]:
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])
    
    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

Create function to mutate a single route

In [ ]:
def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))
            
            city1 = individual[swapped]
            city2 = individual[swapWith]
            
            individual[swapped] = city2
            individual[swapWith] = city1
    return individual

Create function to run mutation over entire population

In [ ]:
def mutatePopulation(population, mutationRate):
    mutatedPop = []
    
    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

Put all steps together to create the next generation

In [ ]:
def nextGeneration(currentGen, eliteSize, mutationRate, migrationInterval,migrationSize, generationCount): #subCount change needed for trying both ga
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)

    return nextGeneration


Proposed migration operator implemented can be found below:

In [ ]:
def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations, subPopSize,migrationInterval,migrationSize):

    a = createSubPopulation(popSize, population,subPopSize) #[][][]
    b = createSubPopulation(popSize, population,subPopSize)
    c = createSubPopulation(popSize, population,subPopSize)
    d = createSubPopulation(popSize, population,subPopSize)

    print("Initial distance: " + str(1 / rankRoutes(a[0])[0][1]))
    print("Initial distance: " + str(1 / rankRoutes(b[0])[0][1]))
    print("Initial distance: " + str(1 / rankRoutes(c[0])[0][1]))
    print("Initial distance: " + str(1 / rankRoutes(d[0])[0][1]))
    
    for (w,x,y,z) in zip(a,b,c,d):
      for i in range(1,generations + 1):
        # print('start of gen', i)
        a = [nextGeneration(a[0], eliteSize, mutationRate, migrationInterval,migrationSize, i)]
        b = [nextGeneration(b[0], eliteSize, mutationRate, migrationInterval,migrationSize, i)]
        c = [nextGeneration(c[0], eliteSize, mutationRate, migrationInterval,migrationSize, i)]
        d = [nextGeneration(d[0], eliteSize, mutationRate, migrationInterval,migrationSize, i)]
        # print('end of gen', i)
        if i % migrationInterval == 0:
          # print('Perform migration')
          
          competitorDict = {} #list of indexes of best route

          competitorDict['b'] = rankRoutes(b[0])[0]
          competitorDict['c'] = rankRoutes(c[0])[0]
          competitorDict['d'] = rankRoutes(d[0])[0]

            
          sortedDict = sorted(competitorDict.items(),key = lambda x: x[1][1],reverse=True)
          #route with maximum fitness ; [0] describes the index position
          winner  = sortedDict[0]
          key = str(winner[0])
          
          if key =='b':
            bestRoute = b[0][0]
          elif key == 'c':
            bestRoute = c[0][0]
          elif key == 'd':
            bestRoute = d[0][0]
          
          # print(bestRoute)
          #print("Before Migration: ",a[0])
          worstRouteIndex = rankRoutesAsc(a[0])[0][0]
          # worstRoute = a[0][worstRouteIndex] #Route in terms of (x,y)
          a[0][worstRouteIndex] = bestRoute
          #print("After Migration: ",a[0])

    #calculate final distance
    # print(a)
    BestDistA = 1 / rankRoutes(a[0])[0][1]
    BestDistB = 1 / rankRoutes(b[0])[0][1]
    BestDistC = 1 / rankRoutes(c[0])[0][1]
    BestDistD = 1 / rankRoutes(d[0])[0][1]
    
    # print("Final distance: " + str(1 / rankRoutes(a[0])[0][1])) #Best individuals from each subpopulation
    # print("Final distance: " + str(1 / rankRoutes(b[0])[0][1]))
    # print("Final distance: " + str(1 / rankRoutes(c[0])[0][1]))
    # print("Final distance: " + str(1 / rankRoutes(d[0])[0][1]))

    finalBest = min(BestDistA,BestDistB,BestDistC,BestDistD)
    print("Final Best distance: " + str(finalBest))


    bestRouteIndex = rankRoutes(a[0])[0][0]
    bestRoute = a[0][bestRouteIndex]
    return bestRoute

## Running the genetic algorithm

Create list of cities

In [ ]:
problem = tsplib95.load('./bier127.tsp') #Load TSPLIB dataset here
print(problem.as_keyword_dict()['NODE_COORD_SECTION'].get(1))
cityList = []
for i in range(1,128):
    x = problem.as_keyword_dict()['NODE_COORD_SECTION'].get(i)[0]
    y = problem.as_keyword_dict()['NODE_COORD_SECTION'].get(i)[1]
    cityList.append(City(x,y))
print(cityList)

[9860, 14152]
[(9860,14152), (9396,14616), (11252,14848), (11020,13456), (9512,15776), (10788,13804), (10208,14384), (11600,13456), (11252,14036), (10672,15080), (11136,14152), (9860,13108), (10092,14964), (9512,13340), (10556,13688), (9628,14036), (10904,13108), (11368,12644), (11252,13340), (10672,13340), (11020,13108), (11020,13340), (11136,13572), (11020,13688), (8468,11136), (8932,12064), (9512,12412), (7772,11020), (8352,10672), (9164,12876), (9744,12528), (8352,10324), (8236,11020), (8468,12876), (8700,14036), (8932,13688), (9048,13804), (8468,12296), (8352,12644), (8236,13572), (9164,13340), (8004,12760), (8584,13108), (7772,14732), (7540,15080), (7424,17516), (8352,17052), (7540,16820), (7888,17168), (9744,15196), (9164,14964), (9744,16240), (7888,16936), (8236,15428), (9512,17400), (9164,16008), (8700,15312), (11716,16008), (12992,14964), (12412,14964), (12296,15312), (12528,15196), (15312,6612), (11716,16124), (11600,19720), (10324,17516), (12412,13340), (12876,12180), (1368

Run the genetic algorithm

In [ ]:
geneticAlgorithm(population = cityList, popSize = 15, eliteSize = 4, mutationRate = 0.01, generations = 500, subPopSize = 4,migrationInterval = 10,migrationSize = 1)


## Plot the progress

Note, this will win run a separate GA

In [ ]:
def geneticAlgorithmPlot(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)
    progress = []
    progress.append(1 / rankRoutes(pop)[0][1])
    
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
        progress.append(1 / rankRoutes(pop)[0][1])
    
    plt.plot(progress)
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    plt.show()

Run the function with  to observe the performance improvement in each generation

In [ ]:
geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=200)